In [1]:
import ipywidgets as widgets
from IPython.display import display ,Javascript
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes



def generate_rsa_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()
    return private_key, public_key

def encrypt_message(message, public_key):
    ciphertext = public_key.encrypt(
        message.encode('utf-8'),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def decrypt_message(ciphertext, private_key):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext.decode('utf-8')


private_key, public_key = generate_rsa_keys()

def handle_generate_keys_button(b):
    private_key, public_key = generate_rsa_keys()
    private_key_text.value = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ).decode()
    public_key_text.value = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ).decode()

def handle_encrypt_button(b):
    message = message_input.value
    ciphertext = encrypt_message(message, public_key)
    ciphertext_output.value = ciphertext.hex()

def handle_decrypt_button(b):
    ciphertext_hex = ciphertext_input.value
    ciphertext = bytes.fromhex(ciphertext_hex)
    decrypted_message = decrypt_message(ciphertext, private_key)
    decrypted_message_output.value = decrypted_message



generate_keys_button = widgets.Button(description="Générer une paire de clés RSA", button_style="success")
generate_keys_button.on_click(handle_generate_keys_button)

encrypt_button = widgets.Button(description="Chiffrer le message", button_style="success")
encrypt_button.on_click(handle_encrypt_button)

decrypt_button = widgets.Button(description="Déchiffrer le message", button_style="success")
decrypt_button.on_click(handle_decrypt_button)

private_key_text = widgets.Textarea(rows=5, description="Clé privée")
public_key_text = widgets.Textarea(rows=5, description="Clé publique")
message_input = widgets.Text()

ciphertext_output = widgets.Textarea(description="Résultat")
ciphertext_input = widgets.Text()
decrypted_message_output = widgets.Textarea(rows=5, description="Résultat")
def handle_copy_button(b):
    js_code = f"navigator.clipboard.writeText('{ciphertext_output.value}')"
    display(Javascript(js_code))
    
copy_button = widgets.Button(description="Copier le résultat", button_style="primary")
copy_button.on_click(handle_copy_button)


# Affichage des éléments dans la cellule de sortie
display(widgets.VBox([generate_keys_button, private_key_text, public_key_text]))
display(widgets.VBox([widgets.Label("Saisir le message à chiffrer :"), message_input, encrypt_button, ciphertext_output,copy_button]))
display(widgets.VBox([widgets.Label("Saisir le message chiffré en hexadécimal :"), ciphertext_input, decrypt_button, decrypted_message_output]))


<IPython.core.display.Javascript object>